In [11]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

layers = keras.layers
regularizers = keras.regularizers
mnist = keras.datasets.mnist

#HYPERPARAMETERS
BATCH_SIZE = 32
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28* 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28* 28).astype("float32") / 255.0

In [42]:
class Dense(layers.Layer):
    def __init__(self, units, input_dim):
        super(Dense, self).__init__()
        self.w = self.add_weight(
            name='w',# without declare a name we can not build the model.
            shape=(input_dim,  units),
            initializer='random_normal',
            trainable=True
        )
        self.b = self.add_weight(
            name= 'b',
            shape=(units,), initializer='zeros',
            trainable=True
        )

    def call(self,inputs):
        return tf.matmul(inputs, self.w)+self.b

In [43]:
class MyModel(keras.Model):
    def __init__(self, num_classes =10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64,784)
        self.dense2 = Dense(num_classes, 64)
        # self.dense1 = layers.Dense(64)
        # self.dense2 = layers.Dense(num_classes)
        
    def call(self, input_tensor):
        x = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)
    
model = MyModel(num_classes=10)
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1, verbose=2)

model.evaluate(x_test, y_test, batch_size= BATCH_SIZE, verbose = 2)

1875/1875 - 12s - loss: 0.3463 - accuracy: 0.9047 - 12s/epoch - 6ms/step
313/313 - 1s - loss: 0.1911 - accuracy: 0.9471 - 1s/epoch - 4ms/step


[0.1910843700170517, 0.9470999836921692]

In [55]:
class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b", shape=(self.units,), initializer="random_normal", trainable=True,
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [56]:
class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

In [57]:
class MyModel(keras.Model):
    def __init__(self, num_classes =10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()
        
    def call(self, input_tensor):
        x = self.relu(self.dense1(input_tensor))
        return self.dense2(x)
    
model = MyModel(num_classes=10)
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1, verbose=2)

model.evaluate(x_test, y_test, batch_size= BATCH_SIZE, verbose = 2)

1875/1875 - 13s - loss: 0.3495 - accuracy: 0.9038 - 13s/epoch - 7ms/step
313/313 - 2s - loss: 0.2062 - accuracy: 0.9406 - 2s/epoch - 6ms/step


[0.2061719298362732, 0.9405999779701233]